Welcome to my plotting program, it is not ready yet. Here I wish to plot plot and colour code a 3D plot. (22/05/2025) -Keely Dodd-Clements 

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import re
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

def format_solvents(solvent_string):
    """
    Convert DMSO(0.28)-ACN(0.13)-EtOH(0.34)-MeTHF(0.25) 
    to formatted HTML string with percentages
    """
    if pd.isna(solvent_string):
        return "No solvents data"
    
    # Parse the solvent string using regex
    pattern = r'([A-Za-z0-9]+)\(([0-9.]+)\)'
    matches = re.findall(pattern, str(solvent_string))
    
    if not matches:
        return str(solvent_string)
    
    formatted_lines = ["<b>Solvents:</b>"]
    for solvent, fraction in matches:
        percentage = float(fraction) * 100
        formatted_lines.append(f"&nbsp;&nbsp;&nbsp;{solvent} ({percentage:.0f}%)")
    
    return "<br>".join(formatted_lines)

def calculate_minimal_sphere(group_df):
    """
    Calculate center of mass and radius using longest 3D distance from center of mass
    """
    if len(group_df) == 1:
        center = [group_df['D'].iloc[0], group_df['P'].iloc[0], group_df['H'].iloc[0]]
        return np.array(center), 0.1  # Small radius for single point
    
    # Calculate center of mass: mean of D, P, H coordinates
    center_d = group_df['D'].mean()
    center_p = group_df['P'].mean() 
    center_h = group_df['H'].mean()
    center_of_mass = np.array([center_d, center_p, center_h])
    
    # Calculate 3D distances from center of mass to each point
    distances = []
    for _, row in group_df.iterrows():
        point = np.array([row['D'], row['P'], row['H']])
        distance = np.linalg.norm(point - center_of_mass)
        distances.append(distance)
    
    # Use the longest 3D distance as radius
    radius = max(distances) if distances else 0.1
    
    # Add small buffer (5% extra)
    radius *= 1.05
    
    return center_of_mass, radius

def create_sphere_mesh(center, radius, resolution=20):
    """Create sphere mesh for plotly"""
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
    
    return x, y, z

def get_solute_info(group_df):
    """Create formatted solute information for sphere hover"""
    info_lines = [f"<b>Solute: {group_df['Solutes'].iloc[0]}</b>"]
    info_lines.append(f"Number of data points: {len(group_df)}")
    
    # Add D, P, H ranges
    d_range = f"D: {group_df['D'].min():.2f} - {group_df['D'].max():.2f}"
    p_range = f"P: {group_df['P'].min():.2f} - {group_df['P'].max():.2f}"
    h_range = f"H: {group_df['H'].min():.2f} - {group_df['H'].max():.2f}"
    
    info_lines.extend([d_range, p_range, h_range])
    
    # Add unique researchers if available
    if 'Researcher' in group_df.columns:
        researchers = group_df['Researcher'].dropna().unique()
        if len(researchers) > 0:
            researcher_list = ", ".join(researchers[:3])  # Show first 3
            if len(researchers) > 3:
                researcher_list += f" (+{len(researchers)-3} more)"
            info_lines.append(f"Researchers: {researcher_list}")
    
    return "<br>".join(info_lines)

# Load the data
file_path = 'PlottedInks.xlsx'
inks_df = pd.read_excel(file_path, sheet_name='Inks')

# Format solvents for hover display
inks_df['formatted_solvents'] = inks_df['Solvents'].apply(format_solvents)

# Create title widget
title = widgets.HTML(
    value="<h2 style='text-align:center;'>Inks Plotted Via Solvents System In Hansen Space, Colour's Indicate Solutes</h2>"
)

# Create toggle button for spheres
sphere_toggle = widgets.ToggleButton(
    value=True,
    description='Show Solute Volumes',
    disabled=False,
    button_style='info',
    tooltip='Toggle translucent spheres around solute groups',
    icon='eye'
)

# Create the main scatter plot
fig = px.scatter_3d(
    inks_df,
    x='D', y='P', z='H',
    color='Solutes',
    custom_data=['formatted_solvents'],
    title='3D Plot of Inks (D, P, H)',
    labels={'D': 'D', 'P': 'P', 'H': 'H'}
)

# Update hover template to show formatted solvents instead of researcher
fig.update_traces(
    hovertemplate="<b>%{fullData.name}</b><br>" +
                  "D: %{x}<br>" +
                  "P: %{y}<br>" +
                  "H: %{z}<br>" +
                  "%{customdata[0]}<br>" +
                  "<extra></extra>"
)

# Group by solutes and add translucent spheres
solute_groups = inks_df.groupby('Solutes')
colors = px.colors.qualitative.Plotly  # Use plotly's color palette
sphere_traces = []  # Store sphere traces for toggling

for i, (solute_name, group) in enumerate(solute_groups):
    if len(group) > 1:  # Only create sphere if more than one point
        # Calculate center of mass and radius for this solute group
        center, radius = calculate_minimal_sphere(group)
        
        # Create sphere mesh
        x_sphere, y_sphere, z_sphere = create_sphere_mesh(center, radius)
        
        # Get color for this solute (cycle through available colors)
        color_idx = i % len(colors)
        sphere_color = colors[color_idx]
        
        # Get solute information for hover
        solute_info = get_solute_info(group)
        
        # Add translucent sphere with custom hover info
        sphere_trace = go.Surface(
            x=x_sphere,
            y=y_sphere,
            z=z_sphere,
            opacity=0.15,
            colorscale=[[0, sphere_color], [1, sphere_color]],
            showscale=False,
            name=f'{solute_name}_envelope',
            legendgroup=solute_name,
            showlegend=False,
            hovertemplate=solute_info + "<extra></extra>",
            visible=True
        )
        
        fig.add_trace(sphere_trace)
        sphere_traces.append(len(fig.data) - 1)  # Store trace index

# Calculate the data range to set equidistant grid
d_range = [inks_df['D'].min(), inks_df['D'].max()]
p_range = [inks_df['P'].min(), inks_df['P'].max()]
h_range = [inks_df['H'].min(), inks_df['H'].max()]

# Find the overall range and create equidistant ticks
all_ranges = [d_range[1] - d_range[0], p_range[1] - p_range[0], h_range[1] - h_range[0]]
max_range = max(all_ranges)
tick_step = max_range / 10  # 10 divisions

# Update layout for better visuals with equal axis scaling and equidistant grid
fig.update_layout(
    legend_title_text='Solutes',
    margin=dict(l=0, r=0, b=0, t=30),
    height=700,
    scene=dict(
        xaxis_title='D',
        yaxis_title='P',
        zaxis_title='H',
        aspectmode='cube',  # Keep proportions cubic
        aspectratio=dict(x=1, y=1, z=1),  # Force equal aspect ratios
        xaxis=dict(
            dtick=tick_step,
            showgrid=True,
            gridwidth=1
        ),
        yaxis=dict(
            dtick=tick_step,
            showgrid=True,
            gridwidth=1
        ),
        zaxis=dict(
            dtick=tick_step,
            showgrid=True,
            gridwidth=1
        )
    )
)

# Create plot widget
plot_widget = go.FigureWidget(fig)

def toggle_spheres(change):
    """Toggle visibility of sphere traces"""
    with plot_widget.batch_update():
        for trace_idx in sphere_traces:
            plot_widget.data[trace_idx].visible = change['new']

# Connect the toggle button to the function
sphere_toggle.observe(toggle_spheres, names='value')

# Create control panel
controls = widgets.HBox([
    sphere_toggle
], layout=widgets.Layout(justify_content='center', margin='10px'))

# Display everything using widgets for Voila
plot_output = widgets.Output()
with plot_output:
    display(plot_widget)

# Combine all elements in a vertical layout
ui = widgets.VBox([
    title, 
    controls,
    plot_output
])

display(ui)